In [ ]:
# !pip install -qU langchain

In [ ]:
# !pip install -U weaviate-client

In [ ]:
from IPython.display import display, Markdown

In [ ]:
import weaviate
from weaviate.classes.init import Auth

In [ ]:
from google.colab import userdata
weaviate_url = userdata.get("weviate_cluster")
weaviate_api_key = userdata.get("weviate_key")

In [ ]:
client=weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

In [ ]:
# !pip install sentence-transformers

In [ ]:
# !pip install -qU langchain-community pypdf

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
pages=PyPDFLoader("/content/student_handbook.pdf",extract_images=True).load()

In [ ]:
# pages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
docs=text_splitter.split_documents(pages)

In [ ]:
# docs

In [ ]:
# !pip install langchain-weaviate

In [ ]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
vector_db = WeaviateVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    client=client,
    by_text=False

    # docs, embeddings, client=client, by_text=False
)

In [ ]:
print(vector_db.similarity_search('what is the registration process in iit bhilai?',k=3)[0].page_content)

parts of this process rests with the students. If a student fails to complete the registration 
process within the specified time, he/she shall be deemed to be on una uthorized absence 
from the Institute and suitable disciplinary action including the termination of the studentship 
shall be carried out by the Institute. 
Academic Pre-registration 
• All students must pre -register for the courses scheduled to be taken in the next 
semester as per schedule specified in the Academic Calendar. 
• It will be the responsibility of the students to ensure that they meet the eligibility criteria, 
pre-requisite criteria if any. Students may only register for the number of courses as 
permitted by semester load rules and must also ensure that there are no timetable 
clashes amongst the courses. The students shall need to submit the fully filled 
registration form  along with the permissions from the instructors if needed  for the 
approval of the DPGC/DUGC.


In [ ]:
print(vector_db.similarity_search('what is the registration process in iit bhilai?',k=3)[1].page_content)

registration must obtain prior permission from Dean of Academic Affairs. They shall be 
treated as non-reporting students if they fail to do so. 
Administrative Registration 
• All students must pay for all dues, including the hostel dues, academic fees, any other 
dues that the Institute may prescribe from time to time etc. during the administrative 
registration. 
• All students whose ID card expires must get it renewed as pe r the norms of the 
Institute. 
• All students must carry out registration of facilities such as various labs, library etc. as 
specified by the Institute during the administrative registration. 
• Any failure to do the administrative registration as per the stipu lated time table shall 
attract penalty, disciplinary action and/or termination of the program. 
• A doctorate student who has submitted the thesis and is waiting for defense shall 
register for zero credits if he/she wishes to stay on campus. The student shall pay all


In [ ]:
from re import template
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say this is not mentioned in the student-handbook.
and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say this is not mentioned in the student-handbook.\nand keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [ ]:
# !pip install -qU langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
huggingface_token=userdata.get("h_key")

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    huggingfacehub_api_token=huggingface_token,
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)

chat_model = ChatHuggingFace(llm=llm)

In [ ]:
chat_model.invoke("what is the registration process in iit bhilai?").content

" I'm here to help you with general information, but please note that the specific details might vary each year or for different institutions. As of now, IIT Bhilai is under construction and not yet operational. However, I can provide you with a general idea of the registration process for IITs based on the current processes in other IITs.\n\n1. Joint Entrance Examination (JEE) Main: To be eligible for admission into IITs, you need to appear for the JEE Main exam conducted by the National Testing Agency (NTA).\n\n2. Joint Seat Allocation Authority (JoSAA): Based on your JEE Main rank, you can participate in the JoSAA counseling process to choose your preferred IIT and course.\n\n3. Registration: During the JoSAA counseling process, you will be required to register on the JoSAA website using your JEE Main roll number and other necessary details.\n\n4. Choice Filling and Locking: After registration, you can fill in your preferred choices of IITs and courses in order of preference. Make s

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

In [ ]:
retriever=vector_db.as_retriever()

In [ ]:
retriever.get_relevant_documents("what is the registration process in iit bhilai?")

[Document(metadata={'source': '/content/student_handbook.pdf', 'creationdate': datetime.datetime(2023, 6, 30, 19, 57, 14, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), 'total_pages': 28.0, 'page': 10.0, 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': datetime.datetime(2023, 6, 30, 19, 57, 14, tzinfo=datetime.timezone(datetime.timedelta(seconds=19800))), 'producer': 'Microsoft® Word for Microsoft 365', 'author': 'IIT BHILAI', 'page_label': '11'}, page_content='parts of this process rests with the students. If a student fails to complete the registration \nprocess within the specified time, he/she shall be deemed to be on una uthorized absence \nfrom the Institute and suitable disciplinary action including the termination of the studentship \nshall be carried out by the Institute. \nAcademic Pre-registration \n• All students must pre -register for the courses scheduled to be taken in the next \nsemester as per schedule specified in the Academic Calendar. \n• It

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | output_parser
)

In [ ]:
res=rag_chain.invoke("tell me the grading system of iit bhilai")

In [ ]:
display(Markdown(res))

 The grading system for regular courses at IIT Bhilai is as follows: A+ (10 points), A (10 points), A– (9 points), B (8 points), B– (7 points), C (6 points), C– (5 points), D (4 points), F (0 points), FS (0 points), I.

In [ ]:
res=rag_chain.invoke("what is the registration process in iit bhilai?")

In [ ]:
display(Markdown(res))

 The registration process in IIT Bhilai involves two steps: Academic Pre-registration and Administrative Registration.

1. Academic Pre-registration: All students must pre-register for the courses scheduled to be taken in the next semester as per the schedule specified in the Academic Calendar. Students must ensure they meet the eligibility criteria, pre-requisite criteria, and there are no timetable clashes amongst the courses. They need to submit the fully filled registration form along with the permissions from the instructors if needed for the approval of the DPGC/DUGC.

2. Administrative Registration: All students must pay for all dues, including the hostel dues, academic fees, any other dues that the Institute may prescribe from time to time, get their ID card renewed if it expires, register for facilities such as various labs, library etc. as specified by the Institute. Failure to do the administrative registration as per the stipulated time table shall attract penalty, disciplinary action, and/or termination of the program.